<a href="https://colab.research.google.com/github/JManuelRG/propiedades_termodinamicas/blob/main/Sustancias%20Puras/Ecuacion_de_Antoine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ecuación de Antoine
# Contantes Obtenidas de la base de datos web de NIST:

https://webbook.nist.gov/chemistry/

# Web scrapper:
https://github.com/oscarcontrerasnavas/NIST-web-book-scraping/blob/master/README.md

https://pypi.org/project/beautifulsoup4/


# Instrucciones:

Ejecutar la siguiente función para obtener los parámetros de la ecuación de Antoine.

La celda es un Web Scrapper que buscará el compuesto en la base de datos de NIST y obtendrá los datos para ese compuesto.

In [1]:
# Importar la función get del módulo requests porque es la función encargada de
# obtener la solicitud HTTP GET con la URL dada.
from requests import get

# Importar BeautifulSoup de bs4 porque realiza el análisis del HTML y nos ayuda a
# manejar el DOM.
from bs4 import BeautifulSoup

# Importar closing para asegurar que cualquier recurso de red se liberará cuando
# salgan del ámbito.
from contextlib import closing

def get_antoine_coef(Name, Temperature):
    global cols
    """ Return a list with the coefficients A, B and C if they exist for the
        given Temperature. If not, return None and print it.

    :param Name:
        A string with the name of the compound in English.

    :param Temperature:
        A float number with the temperature in Kelvin.

    :rtype: List

    :return coef with [A, B, C]

    """

    # Obtener la tabla usando la función get_html mostrada abajo. Table es un
    # objeto BeautifulSoup.
    table = get_html_table(Name)

    # Extraer las filas de la tabla. Sabiendo qué etiquetas tiene una tabla HTML.
    # También, sabiendo que la primera fila con el encabezado de la tabla no tiene
    # el atributo de clase 'exp', por lo que obtenemos solo las filas con datos.
    # La función find_all de BeautifulSoup devuelve una lista.
    rows = table.find_all('tr', class_='exp')

    # Declarar las listas para almacenar Temperatures y coeficientes.
    Temperatures, As, Bs, Cs = [], [], [], []

    # Recorrer las filas para extraer y llenar las variables As, Bs y Cs porque ahora
    # estamos seguros de que la Temperature está en algún rango.
    for row in rows:

        # Al igual que las filas, extraemos las columnas para la fila actual. Sabiendo que
        # las columnas tienen la etiqueta <td> en HTML también.
        # La función find_all de BeautifulSoup devuelve una lista.
        cols =  row.find_all('td')

        # Primero transformar las cadenas en números flotantes y ponerlos en su
        # respectiva lista.
        As.append(float(cols[1].text))
        Bs.append(float(cols[2].text))
        Cs.append(float(cols[3].text))

        # Para las temperaturas, tenemos un rango y necesitamos extraer cada
        # límite (inferior y superior) y ponerlos en una lista adicional. Así que
        # la variable Temperatures será una lista de listas.
        lower_lim = float(cols[0].text.split(' ')[0])
        higher_lim = float(cols[0].text.split(' ')[-1])
        Temperatures.append([lower_lim, higher_lim])


    # Comprobar si la Temperature dada se ajusta a algún intervalo.
    index = None
    for i, interval in enumerate(Temperatures):
        if (interval[0] <= Temperature
            and Temperature <= interval[1]):
            index = i
            break
        else:
            index = None

    if index == None:
        print('Los datos para la temperatura %.2f K no existen en la base de datos' % Temperature)
        return None
    else:
        A = As[index]
        B = Bs[index]
        C = Cs[index]
        return [A, B, C]


def get_html_table(Name):

    """ Return the html already parsed using the a helper function listed below.

    :param Name:
        A string with the name of the compound in English.

    :rtype: BeautifulSoup Object

    """

    # El parámetro name es parte de la URL. Por ejemplo, si quieres los
    # datos del metano, la URL es
    # https://webbook.nist.gov/cgi/cbook.cgi?Name=methane&Mask=4.
    url = str.format('https://webbook.nist.gov/cgi/cbook.cgi?Name={0}&Mask=4', Name.lower())

    # Función para obtener la solicitud realizada, ver abajo.
    raw_html = get_response(url)

    # Analizar el HTML usando BeautifulSoup.
    html = BeautifulSoup(raw_html, 'html.parser')

    # Extraer la tabla que contiene los datos, la tabla tiene un atributo
    # específico 'aria-label' como 'Antoine Equation Parameters'.
    table = html.find('table', attrs={'aria-label': 'Antoine Equation Parameters'})

    return table


def get_response(url):

    """ Return the raw_html for parsing later or None if can't reach the page

    :param url:
        The string for the GET request.

    :rtype: BeautifulSoup Object

    :rtype: None if can't reach the website

    """

    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except:
        print('Not found')
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200
            and content_type is not None
            and content_type.find('html') > -1)



# Forma de la ec. de Antoine

$$log_{10}(P) = A −\frac{B}{T+C}$$

P = Presión de vapor (bar),  T = temperatura (K)

In [5]:
import pandas as pd
# Obtener las variables de la ecuación de Antoine
print("Constantes de la Ec. de Antoine:")
nombre = input('Escribe el nombre del compuesto en inglés: ')
T = float(input('Escribe la temperatura promedio del compuesto, K: '))
A,B,C = get_antoine_coef(nombre,
                  T)
print(f'Tabla de constantes de Antoine para -{nombre}-, válidos a una temperatura de {T} Kelvin')
print('Obtenidos de la base de Datos NIST:')
print(f'URL = https://webbook.nist.gov/cgi/cbook.cgi?Name={nombre}&Mask=4')
datosAntoine = pd.DataFrame({'Compuesto':nombre,
                             'A':[A],
                             'B':[B],
                             'C':[C]})
datosAntoine.set_index('Compuesto')

Constantes de la Ec. de Antoine:
Escribe el nombre del compuesto en inglés: water
Escribe la temperatura promedio del compuesto, K: 403.15
Tabla de constantes de Antoine para -water-, válidos a una temperatura de 403.15 Kelvin
Obtenidos de la base de Datos NIST:
URL = https://webbook.nist.gov/cgi/cbook.cgi?Name=water&Mask=4


,A,B,C
Compuesto,,,
water,3.55959,643.748,-198.043


# Valores Puntuales:

# Cálculo de la Presión de saturación de la sustancia:
# Forma de la ec. de Antoine

$$log_{10}(P) = A −\frac{B}{T+C}$$

P = Presión de vapor (bar),  T = temperatura (K)

$$P^{sat} = 10^{A −\frac{B}{T^{sat}+C}}$$

In [4]:
print('Determinación de la Presión de saturación de un sistema:')
Tsat = float(input('Escribe el valor de la temperatura de saturación en °C: '))
Psat = 10**(datosAntoine['A']-datosAntoine['B']/(Tsat + 273.15 + datosAntoine['C']))
print(f"La presión de saturación de '{datosAntoine['Compuesto'][0]}' calculada, es de {Psat[0]} bar")
print(f"Psat = {Psat[0]*100} kPa")

Determinación de la Presión de saturación de un sistema:
Escribe el valor de la temperatura de saturación en °C: 130
La presión de saturación de 'water' calculada, es de 2.6362952530224657 bar
Psat = 263.6295253022466 kPa


# Cálculo de la Temperatura de Saturación de la sustancia:

$$T(°C) = \frac{B}{(A-log_{10}P(bar))}-C$$

In [6]:
import math
print('Determinación de la temperatura de saturación de un sistema:')
Psatc = float(input('Escribe el valor de la presión de saturación en bar: '))

Tsatc = datosAntoine['B']/(datosAntoine['A']-math.log10(Psatc))-datosAntoine['C']

print(f"La temperatura de saturación de '{datosAntoine['Compuesto'][0]}', calculada es de {Tsatc[0]} K")
print(f"Tsat = {Tsatc[0]-273.15} °C")


Determinación de la temperatura de saturación de un sistema:
Escribe el valor de la presión de saturación en bar: 2.7
La temperatura de saturación de 'water', calculada es de 403.8299065300933 K
Tsat = 130.67990653009332 °C
